# Importing all modules and packages that will be used

In [38]:
import pandas as pd
import numpy as np
# % matplotlib inline

import sqlite3
import pprint
from sqlalchemy import create_engine

# Connecting to the SQLite database

In [39]:
# This code connects to the database through sqlite3
# This enables the database to be queried with traditional SQL code
load = sqlite3.connect('database.sqlite')
cursor = load.cursor()

# This code connects to the database through sqlalchemy
# This enables the database tables to be stored in a pandas dataframe and analyzed with python code and packages
engine = create_engine('sqlite:///database.sqlite')

## Listing all tables in the database

In [41]:
# This code checks the database to list all tables inside
cursor.execute('SELECT name from sqlite_master where type= "table"')
pprint.pprint(cursor.fetchall())

[('sqlite_sequence',),
 ('Player_Attributes',),
 ('Player',),
 ('Match',),
 ('League',),
 ('Country',),
 ('Team',),
 ('Team_Attributes',)]


Even though the above list contains 8 tables, there are actually 7 tables in the database. The first table `sqlite_sequence` is automatically generated by SQLite to keep track of the other tables.

In [42]:
cursor.execute("SELECT * FROM sqlite_sequence")
pprint.pprint(cursor.fetchall())

[('Team', 103916),
 ('Country', 51958),
 ('League', 51958),
 ('Match', 51958),
 ('Player', 11075),
 ('Player_Attributes', 183978),
 ('Team_Attributes', 1458)]


## Creating dataframes for each of the 7 tables

In [ ]:
df_league = pd.read_sql_table("League", engine)
df_player = pd.read_sql_table("Player", engine)
df_match = pd.read_sql_table("Match", engine)
df_country = pd.read_sql_table("Country", engine)
df_team = pd.read_sql_table("Team", engine)
df_team_attr = pd.read_sql_table("Team_Attributes", engine)
df_player_attr = pd.read_sql_table("Player_Attributes", engine)

df = pd.read_sql_table("sqlite_sequence", engine)

df
# c.close()